In [9]:
from cmsis_svd.parser import SVDParser
import json
parser = SVDParser.for_packaged_svd("STMicro", "STM32F446x.svd")
dev = parser.get_device()
peripherals = dev.to_dict()['peripherals']

def reshape_dict(register, periphery):
    register['fields'] = {field['name']:field for field in register['fields']}
    register['address'] = periphery['base_address'] + register['address_offset']
    return register

res = {}
for per in peripherals:
    res[per['name']] = {r['name']:reshape_dict(r, per) for r in per['registers']}

In [22]:
tim2 = Periph('TIM2', res)
tim2.get_modified_regs()

[]

In [44]:
tim10 = Periph("TIM10", res)

In [45]:
tim10.CCER.fields

{'CC1NP': {'name': 'CC1NP',
  'derived_from': None,
  'description': 'Capture/Compare 1 output\n              Polarity',
  'bit_offset': 3,
  'bit_width': 1,
  'access': None,
  'enumerated_values': None,
  'modified_write_values': None,
  'read_action': None},
 'CC1P': {'name': 'CC1P',
  'derived_from': None,
  'description': 'Capture/Compare 1 output\n              Polarity',
  'bit_offset': 1,
  'bit_width': 1,
  'access': None,
  'enumerated_values': None,
  'modified_write_values': None,
  'read_action': None},
 'CC1E': {'name': 'CC1E',
  'derived_from': None,
  'description': 'Capture/Compare 1 output\n              enable',
  'bit_offset': 0,
  'bit_width': 1,
  'access': None,
  'enumerated_values': None,
  'modified_write_values': None,
  'read_action': None}}

In [32]:
tim2.CCER.fields['CC']

{'name': 'CC1E',
 'derived_from': None,
 'description': 'Capture/Compare 1 output\n              enable',
 'bit_offset': 0,
 'bit_width': 1,
 'access': None,
 'enumerated_values': None,
 'modified_write_values': None,
 'read_action': None}

In [16]:
import json


class Register(object):
    '''
    Class, holding description for low-level C-like registers,
    described by desc (generated from SVD file)

        main usage:
        1. create instance
        gpioa = Register(hw_desc['GPIOA']['ODR'])
        2. use set_field to set bit values. This function supports
        both setting and resetting, i.e.
        gpioa.set_field(gpioa.ODR10, 1)
        gpioa.set_field(gpioa.ODR10, 0)

    '''

    def __init__(self, desc):
        self.desc = desc
        for key in desc:
            setattr(self, key, desc[key])

        for field in desc['fields']:
            setattr(self, field, desc['fields'][field])

        self.value = self.reset_value

    def __repr__(self):
        c = "Instance of class Register:\n"
        return c + "{" + "\n".join("{!r}: {!r},"
                               .format(k, v) for k, v in self.desc.items()) + "}"

    def validate(self, offset, bit_width):
        assert offset + bit_width < self.size, \
            "Trying to set bits off register"
        assert "write" in self.access, \
            "Trying to modify read-only register"
        assert ((((2**bit_width) - 1) << offset) & (~self.reset_mask)) == 0, \
            "Trying to set bits on read-only positions"

    def set_bit(self, offset):
        self.validate(offset, 1)
        self.value = self.value | (1 << offset)

    def reset_bit(self, offset):
        self.validate(offset, 1)
        self.value = self.value & ~(1 << offset)

    def set_bits(self, offset, bit_width, value):
        self.validate(offset, bit_width)
        self.value = self.value | (value << offset)

    def reset_bits(self, offset, bit_width):
        self.validate(offset, bit_width)
        self.value = self.value & ~(((2**bit_width) - 1) << offset)

    def set_field(self, field, value):
        offset = field['bit_offset']
        width = field['bit_width']
        self.reset_bits(offset, width)
        self.set_bits(offset, width, value)

    def reset_value(self):
        self.value = self.reset_value


class Periph(object):
    """Base class for uC peripherals"""

    def __init__(self, name, hw_desc):
        super(Periph, self).__init__()
        # with open(hw_desc, "r") as f:
        #    self.hw_desc = json.load(f)
        self.hw_desc = hw_desc
        self.name = name
        self.registers = hw_desc[name]
        regs = {}
        for regname, regdata in self.registers.items():
            reg = Register(regdata)
            setattr(self, regname, reg)
            regs[regname] = reg
        self.registers = regs

    def get_modified_regs(self):
        return [reg for reg in self.registers if reg.value != reg.reset_value]


In [14]:
with open("hw_desc.json", "w") as f:
    json.dump(res, f, indent=4)